In [1]:
# pockyt get -f '{id}  {link}  {tags}  {title} {excerpt}'
import pandas as pd
import bookmarks_parser

bookmarks = bookmarks_parser.parse("/Users/kushalc/Downloads/bookmarks_5_12_20.html")

In [2]:
def _flatten_node(node, parents=()):
    flattened = []
    for child in node.get("children", []):
        try:
            logging.debug("Recursing down: %s <= %s", node["title"], parents)
            flattened += _flatten_node(child, parents + (node["title"],))
        except:
            logging.warn("Couldn't flatten node: %s", node, exc_info=True)
    try:
        if node.get("type") == "bookmark":
            node["parents"] = parents  # .apply(" ≫ ".join)
            flattened.append(node)
    except:
        logging.warn("Couldn't extract node: %s", node, exc_info=True)
    return flattened

flattened_df = pd.DataFrame(_flatten_node(bookmarks[0]) + _flatten_node(bookmarks[1]))
flattened_df["add_date"] = pd.to_datetime(flattened_df["add_date"].astype(int), unit="s")
flattened_df["parents"] = flattened_df["parents"].apply(lambda x: x[1:])
flattened_df = flattened_df.sort_values(["parents", "add_date"], ascending=[True, False])

In [3]:
flattened_df

,type,url,title,add_date,icon,parents
0,bookmark,https://feedly.com/i/my,Feedly,2020-04-15 21:28:42,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",()
1,bookmark,https://search.firstround.com/,FirstSearch,2020-02-08 09:35:14,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",()
2,bookmark,https://distill.pub/,Distill,2020-01-24 22:59:42,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","(Blogs,)"
15,bookmark,https://eng.uber.com/,Uber Engineering Blog,2020-04-14 21:52:36,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","(Blogs, Engineering)"
12,bookmark,https://eng.lyft.com/,Lyft Engineering,2020-04-14 21:52:09,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","(Blogs, Engineering)"
...,...,...,...,...,...,...
4133,bookmark,http://www.youtube.com/watch?v=yfRVCaA5o18&fea...,YouTube - The Strange Powers of the Placebo Ef...,2016-11-22 09:02:30,None,"(Uncategorized,)"
4134,bookmark,https://www.youtube.com/watch?v=y0mxz2-AQ64&fe...,YouTube - The Wire - How To Play Chess,2016-11-22 09:02:30,None,"(Uncategorized,)"
4135,bookmark,https://m.reddit.com/r/YouShouldKnow/comments/...,YSK the 2x10 method of breaking through to peo...,2016-11-22 09:02:30,None,"(Uncategorized,)"
4138,bookmark,https://en.m.wikipedia.org/wiki/Zoo_hypothesis,"Zoo hypothesis - Wikipedia, the free encyclopedia",2016-11-22 09:02:30,None,"(Uncategorized,)"


In [4]:
flattened_df["tags"] = flattened_df["parents"].str.join(",").str.lower()
flattened_df.to_csv("importable.tsv", sep="\t", columns=["url", "add_date", "title", "tags"], header=False, index=False)